# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

# Good Luck!

In [1]:
import findspark

In [2]:
findspark.init('/home/i-sip_iot/spark-3.0.1-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('DTRF').getOrCreate()

In [6]:
df = spark.read.csv('dog_food.csv', inferSchema=True, header=True)

In [7]:
df.show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
|  9|  8|11.0|  3|    1.0|
|  1| 10|12.0|  3|    1.0|
|  1|  5|13.0| 10|    1.0|
|  2| 10|12.0|  6|    1.0|
|  1| 10|11.0|  4|    1.0|
|  5|  3|12.0|  2|    1.0|
|  4|  9|11.0|  8|    1.0|
|  5|  1|11.0|  1|    1.0|
|  4|  9|12.0| 10|    1.0|
|  5|  8|10.0|  9|    1.0|
+---+---+----+---+-------+
only showing top 20 rows



In [8]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler

In [9]:
assemble = VectorAssembler(inputCols=['A','B','C','D'], outputCol='features')

In [10]:
dfn = assemble.transform(df)

In [11]:
dfn = dfn.select(['features', 'Spoiled'])

In [12]:
RFModel = RandomForestClassifier(featuresCol='features', labelCol='Spoiled', predictionCol='prediction')

In [13]:
# assemble_2 = VectorAssembler(inputCols=['A','B','C','D'], outputCol='features')
# RFModel_A = RandomForestClassifier(featuresCol='A', labelCol='Spoiled', predictionCol='prediction')
# dfn_A = df.select(['A', 'Spoiled'])
# df_train_A, df_test_A = dfn_A.randomSplit([0.7, 0.3])
# trained_model_A = RFModel_A.fit(df_train_A)
# test_A = trained_model_A.transform(df_test_A)

In [14]:
df_train, df_test = dfn.randomSplit([0.7, 0.3])

In [15]:
trained_model = RFModel.fit(df_train)

In [16]:
test = trained_model.transform(df_test)

In [17]:
test.show()

+-------------------+-------+--------------------+--------------------+----------+
|           features|Spoiled|       rawPrediction|         probability|prediction|
+-------------------+-------+--------------------+--------------------+----------+
|  [1.0,2.0,9.0,1.0]|    0.0|[19.6093770652201...|[0.98046885326100...|       0.0|
|[1.0,6.0,11.0,10.0]|    1.0|[1.47311086596800...|[0.07365554329840...|       1.0|
| [1.0,7.0,11.0,9.0]|    1.0|[1.72311086596800...|[0.08615554329840...|       1.0|
| [1.0,8.0,7.0,10.0]|    0.0|[18.6322674952612...|[0.93161337476306...|       0.0|
|  [2.0,1.0,8.0,9.0]|    0.0|[18.6143832995940...|[0.93071916497970...|       0.0|
|  [2.0,2.0,9.0,8.0]|    0.0|[19.0268782120889...|[0.95134391060444...|       0.0|
|  [2.0,3.0,6.0,9.0]|    0.0|[18.9486828165022...|[0.94743414082511...|       0.0|
| [2.0,3.0,7.0,10.0]|    0.0|[18.6153494831689...|[0.93076747415844...|       0.0|
| [2.0,4.0,11.0,8.0]|    1.0|[1.72056684112246...|[0.08602834205612...|       1.0|
|  [

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [19]:
evalu = BinaryClassificationEvaluator(labelCol='Spoiled')

In [20]:
accu = evalu.evaluate(test)

In [21]:
accu

0.9828106852497096

In [31]:
assemble_a = VectorAssembler(inputCols=['A'], outputCol='features')
dfn_a = assemble_a.transform(df)
unlabled_data = dfn_a.select('features')

In [32]:
pred_a = trained_model.transform(unlabled_data)

In [33]:
pred_a

Py4JJavaError: An error occurred while calling o431.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 33.0 failed 1 times, most recent failure: Lost task 0.0 in stage 33.0 (TID 30, 192.168.2.27, executor driver): org.apache.spark.SparkException: Failed to execute user defined function(ProbabilisticClassificationModel$$Lambda$3058/86004960: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:729)
	at org.apac